# Notebook de Análise de Dados - Trackie

Este notebook oferece uma maneira interativa de explorar e analisar os dados gerados pela aplicação, começando pelo banco de dados de rostos conhecidos.

**Objetivos:**
1.  Listar todas as pessoas cadastradas no banco de dados.
2.  Contar quantas imagens de referência existem para cada pessoa.
3.  Visualizar as imagens de referência salvas para cada pessoa cadastrada.

In [ ]:
import sys
import os
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np # Importado para manipulação de arrays no plot

# --- Configuração de Caminhos ---
# Assume que o notebook está em um diretório como 'notebooks/' na raiz do projeto.
# Ex: /caminho/para/projeto/notebooks/este_notebook.ipynb
try:
    project_root = Path(os.getcwd()).parent
    src_path = project_root / "src"
    if str(src_path) not in sys.path:
        sys.path.insert(0, str(src_path))

    # Importa as configurações para obter o caminho do banco de dados
    from trackie.config.settings import settings
    
    # Caminho para o banco de dados de rostos
    DB_PATH = project_root / settings.paths.data / "user_data" / "known_faces"
    print(f"Analisando o banco de dados de rostos em: {DB_PATH}")

except (ImportError, AttributeError, FileNotFoundError):
    print("Erro: Não foi possível importar as configurações ou encontrar os diretórios do projeto.")
    print("Verifique se o notebook está na pasta correta ('notebooks/') e se a estrutura do projeto está intacta.")
    DB_PATH = None # Define como None para que as células seguintes não executem com erro

## 1. Análise do Banco de Dados de Rostos

Vamos escanear o diretório de dados para identificar todas as pessoas cadastradas e contar o número de imagens de referência para cada uma.

In [ ]:
face_data = []
# Verifica se o caminho foi definido corretamente e se o diretório existe
if DB_PATH and DB_PATH.exists():
    # Itera sobre cada pasta de pessoa no banco de dados
    for person_dir in DB_PATH.iterdir():
        if person_dir.is_dir():
            person_name = person_dir.name
            # Conta apenas os arquivos de imagem .jpg, .png e .jpeg
            image_extensions = ["*.jpg", "*.jpeg", "*.png"]
            image_files = []
            for ext in image_extensions:
                image_files.extend(list(person_dir.glob(ext)))
            
            image_count = len(image_files)
            face_data.append({"Pessoa": person_name, "Nº de Imagens": image_count})
else:
    print("O diretório do banco de dados de rostos não foi encontrado ou não existe.")

# Cria um DataFrame do Pandas para uma visualização clara
if face_data:
    df = pd.DataFrame(face_data)
    # Ordena o DataFrame por nome para uma visualização mais organizada
    df = df.sort_values(by="Pessoa").reset_index(drop=True)
    
    print("Resumo do Banco de Dados de Rostos:")
    # Usa to_string para garantir que todas as linhas sejam exibidas sem truncamento
    print(df.to_string(index=False))
else:
    print("Nenhuma pessoa encontrada no banco de dados.")

## 2. Visualização das Imagens de Referência

Agora, vamos visualizar as imagens de referência para cada pessoa encontrada no banco de dados.

In [ ]:
# Verifica se o DataFrame foi criado na célula anterior
if 'df' in locals() and not df.empty:
    
    # Itera sobre cada pessoa no DataFrame para criar um gráfico para cada uma
    for index, row in df.iterrows():
        person_to_show = row['Pessoa']
        person_path = DB_PATH / person_to_show
        
        # Coleta todos os tipos de imagem
        image_extensions = ["*.jpg", "*.jpeg", "*.png"]
        image_files = []
        for ext in image_extensions:
            image_files.extend(list(person_path.glob(ext)))
        
        if not image_files:
            print(f"\nNenhuma imagem encontrada para {person_to_show}, pulando.")
            continue # Pula para a próxima pessoa
        
        # --- Configuração do Gráfico (Plot) ---
        num_images = len(image_files)
        cols = 4  # Define um máximo de 4 colunas por linha
        rows = (num_images + cols - 1) // cols
        
        fig, axes = plt.subplots(rows, cols, figsize=(15, 4 * rows))
        
        # Garante que 'axes' seja sempre um array iterável, mesmo com 1 imagem
        axes = np.atleast_1d(axes).flatten()
        
        fig.suptitle(f"Imagens de Referência para: {person_to_show} ({num_images} imagens)", fontsize=16)
        
        for i, img_path in enumerate(image_files):
            img = cv2.imread(str(img_path))
            # Converte a imagem de BGR (padrão do OpenCV) para RGB (padrão do Matplotlib)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            axes[i].imshow(img_rgb)
            axes[i].set_title(img_path.name, fontsize=9)
            axes[i].axis('off') # Remove os eixos (x, y)
            
        # Oculta os subplots que não foram utilizados
        for j in range(num_images, len(axes)):
            axes[j].axis('off')
            
        plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Ajusta o layout para evitar sobreposição
        plt.show()
else:
    print("Não há dados para visualizar. Execute a célula de análise primeiro.")